In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

## Inspection score vs number of fatalities

In addition to incident type, we will also look to see if there is a relationship between zipcodes with a high proportion of fires resulting in fatalities and zipcodes with low mean inspection scores.

We'll group our SQL query by the OTH_DEATH column (which indicates non-fire fighter fatality), and get the row counts for each group.

In [3]:
WORKING_DIR = 'D:/Fire Project/data/'

In [4]:
conn = sqlite3.Connection('fire_data.db')
pd.set_option('display.max_columns', None)
result = pd.read_sql("""
    WITH modified_table AS (
    SELECT t1.*,
            COALESCE(t1.OTH_DEATH, 0) AS OTH_DEATH_MODIFIED,
            t2.ZIP5
        FROM basic_incident t1
        JOIN incident_address t2 
            ON t1.INCIDENT_KEY = t2.INCIDENT_KEY
        )
        SELECT
        CASE
            WHEN OTH_DEATH_MODIFIED = 0 THEN '0'
            WHEN OTH_DEATH_MODIFIED = 1 THEN '1'
            WHEN OTH_DEATH_MODIFIED = 2 THEN '2'
            WHEN OTH_DEATH_MODIFIED = 3 THEN '3'
            ELSE 'more than 3'
        END AS OTH_DEATH_GROUP,
        ZIP5 as zip,
        COUNT(*) AS ROW_COUNT
    FROM modified_table
    GROUP BY OTH_DEATH_GROUP, ZIP5
    ORDER BY OTH_DEATH_GROUP, ZIP5
""", conn)

Then we'll convert the raw counts to proportions, so that every row in our resulting DataFrame is a unique zipcode, and the counts of different levels of fatality counts are combined to form proportions.

In [5]:
# Group the DataFrame by ZIP5 and OTH_DEATH_GROUP, summing the ROW_COUNT
grouped_df = result.groupby(["zip", "OTH_DEATH_GROUP"]).sum()

# Unstack the DataFrame to get unique ZIP5 as index and OTH_DEATH_GROUP as columns
unstacked_df = grouped_df.unstack(level=-1)

unstacked_df.fillna(0, inplace=True)
row_sums = unstacked_df.sum(axis=1)

# Calculate proportions by dividing each row by the row sum
proportions_df = unstacked_df.div(row_sums, axis=0)

proportions_df.columns = [f'proportion_DEATH_{col[1]}' for col in proportions_df.columns]
proportions_df = proportions_df.reset_index()

We'll remove the bad zipcode values from the NFIRS dataset (which are non-numeric or not 5-digits).

In [6]:
valid_zip_mask = proportions_df['zip'].astype(str).str.match(r'^\d{5}$')
proportions_df = proportions_df[valid_zip_mask]

We'll load the most recent REAC scores for public and multifamily housing into DataFrames.

In [7]:
multifamily = pd.read_excel(WORKING_DIR + 'multifamily_physical_inspection_scores_0321.xlsx')
public = pd.read_excel(WORKING_DIR + 'public_housing_physical_inspection_scores_0321.xlsx')

Combine our two sets of REAC scores into one DataFrame.

In [8]:
reac = pd.concat([public, multifamily])

Overall we have 1818 properties with 333 being public housing and the rest being multifamily.

Let's get the average REAC score for each zipcode. We'll rename the index and cast it to int for the sake of compatibility with the NFIRST values.

In [9]:
mean_scores = reac.groupby(by='ZIPCODE').INSPECTION_SCORE.mean()
mean_scores = mean_scores.rename_axis('zip')
mean_scores.index = mean_scores.index.astype(int)
mean_scores.sample()

zip
39212    84.25
Name: INSPECTION_SCORE, dtype: float64

Now we can merge the NFIRS fatalities DataFrame with the mean_scores from earlier, which contains the average REAC score in every zipcode in the REAC dataset.

In [10]:
# Cast to int for compatability
mean_scores.index = mean_scores.index.astype(int)

fatalities_scores = pd.merge(proportions_df, mean_scores, how='inner', on='zip')

Now we can plot a coorelation matrix with the results.

In [11]:
fatalities_scores.corr().INSPECTION_SCORE

proportion_DEATH_0             -0.001484
proportion_DEATH_1             -0.003542
proportion_DEATH_2              0.011375
proportion_DEATH_3             -0.014722
proportion_DEATH_more than 3   -0.002307
INSPECTION_SCORE                1.000000
Name: INSPECTION_SCORE, dtype: float64